In [ ]:
import pandas as pd
import numpy as np
import json
from scipy import sparse as sp
from tqdm.notebook import tqdm
from collections import defaultdict

In [ ]:
import sys
sys.path.append('../')

from src.utils import get_shard_path
from src.metrics import normalized_average_precision

In [ ]:
cnt = defaultdict(int)

def read_part(path):
    for js in tqdm((json.loads(s) for s in open(path))):
        for trans in js['transaction_history']:
            for product in trans["products"]:
                cnt[product["product_id"]] += 1

for i in range(4):
    read_part(get_shard_path(i))

_tmp = list(cnt.keys())
top_products = sorted(_tmp, key=lambda x: -cnt[x])

# Check MAP@30

In [ ]:
scores = []
for js in tqdm((json.loads(s) for s in open(get_shard_path(7)))):
    recommended_items = top_products[:30]
    gt_items = js["target"][0]["product_ids"]
    ap = normalized_average_precision(gt_items, recommended_items)
    scores.append(ap)
np.mean(scores)

In [ ]:
! mkdir -p ../tmp/global_top/

In [ ]:
import pickle
pickle.dump(cnt, open('../tmp/global_top/global_counters.pkl', "wb"))